In [19]:
import numpy as np
import pandas as pd

import matplotlib
matplotlib.use('Agg')  
import matplotlib.pyplot as plt

import json
import tweepy
import time

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

In [20]:
from config import consumer_key, consumer_secret, access_token, access_token_secret

In [25]:
def AnalyzeTweets():

    # Tweepy Authentication
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)
    api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())
    mentions = api.search(q='@plotbot5 Analyze:')
 
    # Run a search for the command term
    mentions = api.search(q='@plotbot5 Analyze:')
    content = []
    target_account = ''

    # Grab the most recent command tweet
    command = mentions['statuses'][0]['text']
    requesting_user = mentions['statuses'][0]['user']['screen_name']

    # Split it to determine the target account
    try:
        content = command.split('Analyze:')
        target_account = content[1].strip()

        # Confirm target_account
        print('Target Account: ' + target_account)
        print('Requesting User: ' + requesting_user)

    except Exception:
        raise

    #Avoid Repeated content

    # Grab Self Tweets
    tweets = api.user_timeline()

    # Confirm the target account has never been tweeted before
    repeat = False

    for tweet in tweets:
        if target_account in tweet['text']:
            repeat = True
            print('Repeat Error!')

        else:
            continue

    #sentiment analysis
    if not (repeat):

        tweet_data = {
            'tweet_source': [],
            'tweet_text': [],
            'tweet_date': [],
            'tweet_vader_score': [],
            'tweet_neg_score': [],
            'tweet_pos_score': [],
            'tweet_neu_score': []
        }

        # Grab 500 tweets from the target source
        for x in range(25):

            # Grab the tweets
            tweets = api.user_timeline(target_account, page=x)

            # For each tweet store it into the dictionary
            for tweet in tweets:

                tweet_data['tweet_source'].append(tweet['user']['name'])
                tweet_data['tweet_text'].append(tweet['text'])
                tweet_data['tweet_date'].append(tweet['created_at'])

                tweet_data['tweet_vader_score'].append(
                    analyzer.polarity_scores(tweet['text'])['compound'])
                tweet_data['tweet_pos_score'].append(analyzer.polarity_scores(
                    tweet['text'])['pos'])
                tweet_data['tweet_neg_score'].append(analyzer.polarity_scores(
                    tweet['text'])['neg'])
                tweet_data['tweet_neu_score'].append(analyzer.polarity_scores(
                    tweet['text'])['neu'])

    if not (repeat):

         tweet_data = {
            'tweet_source': [],
            'tweet_text': [],
            'tweet_date': [],
            'tweet_vader_score': [],
            'tweet_pos_score': [],
            'tweet_neg_score': [],
            'tweet_neu_score': []
        }
        
        for x in range(25):

            tweets = api.user_timeline(target_account, page=x)
            
            for tweet in tweets:

                tweet_data['tweet_source'].append(tweet['user']['name'])
                tweet_data['tweet_text'].append(tweet['text'])
                tweet_data['tweet_date'].append(tweet['created_at'])

                # Run sentiment analysis on each tweet using Vader
                tweet_data['tweet_vader_score'].append(
                    analyzer.polarity_scores(tweet['text'])['compound'])
                tweet_data['tweet_pos_score'].append(analyzer.polarity_scores(
                    tweet['text'])['pos'])
                tweet_data['tweet_neg_score'].append(analyzer.polarity_scores(
                    tweet['text'])['neg'])
                tweet_data['tweet_neu_score'].append(analyzer.polarity_scores(
                    tweet['text'])['neu'])

        # Store the final contents into a DataFrame
        tweet_df = pd.DataFrame(tweet_data, columns=['tweet_source',
                                                     'tweet_text',
                                                     'tweet_date',
                                                     'tweet_vader_score',
                                                     'tweet_pos_score',
                                                     'tweet_neu_score',
                                                     'tweet_neg_score'])

        tweet_df.head()

    if not (repeat):

        # must convert data types*****
        tweet_df['tweet_date'] = pd.to_datetime(tweet_df['tweet_date'])

        # Sort the dataframe by date
        tweet_df.sort_values('tweet_date', inplace=True)
        #Drop index
        tweet_df.reset_index(drop=True, inplace=True)
        tweet_df.head()

    if not (repeat):
        plt.clf()

        # Build scatter 
        plt.plot(np.arange(-len(tweet_df['tweet_vader_score']), 0, 1),
                 tweet_df['tweet_vader_score'], marker='o', linewidth=0.5,
                 alpha=0.8, label='%s' % target_account)

        # Incorporate the other graph properties
        plt.title('Sentiment Analysis of Tweets (%s)' % time.strftime('%x'))
        plt.ylabel('Tweet Polarity')
        plt.xlabel('Tweets Ago')
        plt.grid(True)
        lgnd = plt.legend(loc='best', title='Tweets', labelspacing=0.5)
        
        file_path = 'analysis/' + target_account + '.png'
        plt.savefig(file_path)

    if not (repeat):

        # Tweet out the image and mention the user who requested it
        api.update_with_media(file_path,
                              'New Tweet Analysis: %s (Thx @%s!!)' %
                              (target_account, requesting_user))

while(True):
    AnalyzeTweets()
    time.sleep(300)


IndentationError: unindent does not match any outer indentation level (<tokenize>, line 95)